In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

# find_lr function
import math
import matplotlib.pyplot as plt

# DDP
import torch.distributed as dist

### Automatic Mixed Precision (AMP)

In [2]:
from torch.cuda.amp import autocast, GradScaler

### Tensorboard

In [3]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(log_dir="/data/runs/")

### Weights and Biases (wandb)

In [4]:
# !pip install wandb
import wandb
wandb.login()

wandb: Currently logged in as: rubyhan (use `wandb login --relogin` to force relogin)


True

In [5]:
GPU=0

In [6]:
SEED=1

In [7]:
global_step=0

In [8]:
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [9]:
torch.cuda.device_count()

1

In [10]:
START_EPOCH = 0

In [11]:
ARCH = 'resnet18'
# ARCH = 'resnet152'
EPOCHS = 2 #200
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
PRINT_FREQ = 50
TRAIN_BATCH=500 #128
VAL_BATCH=500 #128
WORKERS=2
TRAINDIR="/data/hw9/train"
VALDIR="/data/hw9/val"

In [12]:
wandb.init(project='hw9_main', 
           entity='rubyhan', 
           config = {
               "learning_rate": LR,
               "epochs": EPOCHS,
               "batch_size": TRAIN_BATCH,
               "momentum": MOMENTUM, 
               "weight_decay": WEIGHT_DECAY,
               "architecture": ARCH
           })


CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



In [13]:
# DDP
WORLD_SIZE = 2
BACKEND = 'nccl'
URL = 'tcp://172.31.26.78:1234'
RANK = 0

dist.init_process_group(backend = BACKEND, init_method= URL,
                        world_size= WORLD_SIZE, rank=RANK)

In [14]:
if not torch.cuda.is_available():
    print('GPU not detected.. did you pass through your GPU?')

In [15]:
torch.cuda.set_device(GPU)

In [16]:
cudnn.benchmark = True

In [17]:
def train(train_loader, model, criterion, optimizer, epoch):
    global global_step
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    scaler = GradScaler()
    
    # switch to train mode
    model.train()

    end = time.time()
    
    
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        optimizer.zero_grad()
        
        if GPU is not None:
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(GPU, non_blocking=True)

#         # compute output
#         output = model(images)
#         loss = criterion(output, target)
        
        # WITH AMP
        with autocast():
            output = model(images)
            loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

#         # compute gradient and do SGD step
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
        
        # use the scaler
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        writer.add_scalar("Loss/train", loss, global_step = global_step)
        writer.add_scalar("acc1/train", top1.avg, global_step = global_step)
        writer.add_scalar("acc5/train", top5.avg, global_step = global_step)
        global_step += 1
        
        wandb.log({"Loss/train": loss, "acc1/train": top1.avg, "acc5/train": top5.avg})
        
        if i % PRINT_FREQ == 0:
            progress.display(i)

In [18]:
def validate(val_loader, model, criterion):
    global global_step
    
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if GPU is not None:
                images = images.cuda(GPU, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(GPU, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))
    
    writer.add_scalar("Loss/val", losses.avg, global_step = global_step)
    writer.add_scalar("acc1/val", top1.avg, global_step = global_step)
    writer.add_scalar("acc5/val", top5.avg, global_step = global_step)    
    
    wandb.log({"Loss/val": losses.avg, 'acc1/val': top1.avg, 'acc5/val': top5.avg})
    
    return top1.avg

In [19]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [20]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [21]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [22]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [23]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [24]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]
cinic_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
cinic_std_RGB = [0.24205776, 0.23828046, 0.25874835]
cifar_mean_RGB = [0.4914, 0.4822, 0.4465]
cifar_std_RGB = [0.2023, 0.1994, 0.2010]

In [25]:
normalize = transforms.Normalize(mean=imagenet_mean_RGB, 
                                 std=imagenet_std_RGB)

In [26]:
IMG_SIZE = 224

In [27]:
NUM_CLASSES = 1000

In [28]:
model = models.__dict__[ARCH]()

In [29]:
inf = model.fc.in_features

In [30]:
model.fc = nn.Linear(inf, NUM_CLASSES)

In [31]:
model.cuda(GPU)

# Set model on BOTH GPUs
model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[GPU])

In [32]:
criterion = nn.CrossEntropyLoss().cuda(GPU)

In [33]:
optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)

In [34]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [35]:
transform_train = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [36]:
train_dataset = datasets.ImageFolder(
    TRAINDIR, transform=transform_train)

In [37]:
transform_val = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [38]:
val_dataset = datasets.ImageFolder(
    VALDIR, transform=transform_val)

In [39]:
# train_loader = torch.utils.data.DataLoader(
#         train_dataset, batch_size=TRAIN_BATCH, shuffle=True,
#         num_workers=WORKERS, pin_memory=True, sampler=None)

# adding DDSampler to loader
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=TRAIN_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=torch.utils.data.distributed.DistributedSampler(train_dataset))

In [40]:
val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=VAL_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=None)

In [41]:
best_acc1 = 0

In [42]:
scaler = GradScaler()

def find_lr(init_value = 1e-8, final_value=10., beta = 0.98):
    num = len(train_loader)-1
    mult = (final_value / init_value) ** (1/num)
    lr = init_value
    optimizer.param_groups[0]['lr'] = lr
    avg_loss = 0.
    best_loss = 0.
    batch_num = 0
    losses = []
    log_lrs = []
    
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    meter_losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, meter_losses, top1, top5])
    
    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        batch_num += 1
        # measure data loading time
        data_time.update(time.time() - end)

        if GPU is not None:
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(GPU, non_blocking=True)
        
        with autocast():
            # compute output
            output = model(images)
            loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        meter_losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))
        
        #Compute the smoothed loss
        avg_loss = beta * avg_loss + (1-beta) * loss.item()
        smoothed_loss = avg_loss / (1 - beta**batch_num)
        #Stop if the loss is exploding
        if batch_num > 1 and smoothed_loss > 4 * best_loss:
            return log_lrs, losses
        #Record the best loss
        if smoothed_loss < best_loss or batch_num==1:
            best_loss = smoothed_loss
            
        #Store the values
        losses.append(smoothed_loss)
        log_lrs.append(math.log10(lr))
        
        # compute gradient and do SGD step
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        #Update the lr for the next step
        lr *= mult
        optimizer.param_groups[0]['lr'] = lr
        
        if i % PRINT_FREQ == 0:
            progress.display(i)
        
    return log_lrs, losses

# logs,losses = find_lr()
# plt.plot(logs[10:-5],losses[10:-5])

In [43]:
%%time

# start = time.time()
for epoch in range(START_EPOCH, EPOCHS):
#     adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))
    
    writer.add_scalar("lr", scheduler.get_last_lr()[0], global_step = global_step)
    
    wandb.log({'lr': scheduler.get_last_lr()[0]})
    
# print(f'{time.time() - start:.2f}s')

Epoch: [0][   0/1282]	Time 15.347 (15.347)	Data  3.942 ( 3.942)	Loss 7.0066e+00 (7.0066e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.60 (  0.60)
Epoch: [0][  50/1282]	Time  0.695 ( 1.826)	Data  0.003 ( 0.938)	Loss 6.7108e+00 (6.8437e+00)	Acc@1   0.60 (  0.48)	Acc@5   2.40 (  1.63)
Epoch: [0][ 100/1282]	Time  0.672 ( 1.738)	Data  0.002 ( 0.949)	Loss 6.1412e+00 (6.6136e+00)	Acc@1   2.00 (  0.94)	Acc@5   6.40 (  3.13)
Epoch: [0][ 150/1282]	Time  0.702 ( 1.703)	Data  0.002 ( 0.941)	Loss 5.8201e+00 (6.4211e+00)	Acc@1   3.80 (  1.39)	Acc@5  10.40 (  4.64)
Epoch: [0][ 200/1282]	Time  0.692 ( 1.681)	Data  0.002 ( 0.937)	Loss 5.6502e+00 (6.2552e+00)	Acc@1   4.60 (  1.98)	Acc@5  12.60 (  6.31)
Epoch: [0][ 250/1282]	Time  0.678 ( 1.671)	Data  0.003 ( 0.937)	Loss 5.3398e+00 (6.1112e+00)	Acc@1   5.20 (  2.52)	Acc@5  15.80 (  7.88)
Epoch: [0][ 300/1282]	Time  0.680 ( 1.665)	Data  0.002 ( 0.936)	Loss 5.1088e+00 (5.9808e+00)	Acc@1   9.60 (  3.14)	Acc@5  21.40 (  9.47)
Epoch: [0][ 350/1282]	Time  0.711 ( 1.662

In [44]:
writer.close()
%reload_ext tensorboard
%tensorboard --logdir=/data/runs